In [2]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [4]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']

df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)
df_pcGroup = df_pc.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pcGroup.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pcGroup['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)
df_pcGroup

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
coordinates.rename(columns={"Postal Code": "Postcode"}, inplace=True)
coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
toronto_df_new = df_pcGroup.merge(coordinates, on="Postcode", how="left")
toronto_df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
column_names = ["Postcode", "Borough", "Neighbourhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["Postcode"]==postcode], ignore_index=True)
    
test_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## get the geographical coordinates of Toronto

In [21]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


## visualize Manhattan

In [26]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [27]:
import folium

In [29]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare Credentials and Version

In [117]:
CLIENT_ID = 'YM0FQLFGKJ4XJGG4YNGTZ0AFUVIE3YQVRTBOAJ4I5IBYKKZR' 
CLIENT_SECRET = 'URKCO5BBXOIV1W5Y0G2AVU10BKDKELKIAXXN23AUVQW0PZXU' 
VERSION = '20180605'

In [118]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Postcode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=YM0FQLFGKJ4XJGG4YNGTZ0AFUVIE3YQVRTBOAJ4I5IBYKKZR&client_secret=URKCO5BBXOIV1W5Y0G2AVU10BKDKELKIAXXN23AUVQW0PZXU&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


## create a new dataframe called venues

In [119]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['Postcode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4429, 9)


,Postcode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Cafe Plenty,43.654571,-79.389450,Café
3,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
4,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Japango,43.655268,-79.385165,Sushi Restaurant


In [120]:
venues_df.groupby('Neighbourhood').count()


,Postcode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,,,
Agincourt,43,43,43,43,43,43,43,43
"Alderwood, Long Branch",43,43,43,43,43,43,43,43
"Bathurst Manor, Wilson Heights, Downsview North",43,43,43,43,43,43,43,43
Bayview Village,43,43,43,43,43,43,43,43
"Bedford Park, Lawrence Manor East",43,43,43,43,43,43,43,43
Berczy Park,43,43,43,43,43,43,43,43
"Birch Cliff, Cliffside West",43,43,43,43,43,43,43,43
"Brockton, Parkdale Village, Exhibition Place",43,43,43,43,43,43,43,43
Business reply mail Processing Centre,43,43,43,43,43,43,43,43


## analyzing each neighbourhood

In [121]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-1:]) + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(4429, 34)


,Neighbourhood,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Bistro,Breakfast Spot,Bubble Tea Shop,Café,Coffee Shop,...,Park,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Malvern, Rouge",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [122]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Bistro,Breakfast Spot,Bubble Tea Shop,Café,Coffee Shop,...,Park,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,Agincourt,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
1,"Alderwood, Long Branch",0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
2,"Bathurst Manor, Wilson Heights, Downsview North",0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
3,Bayview Village,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
4,"Bedford Park, Lawrence Manor East",0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
5,Berczy Park,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
6,"Birch Cliff, Cliffside West",0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
7,"Brockton, Parkdale Village, Exhibition Place",0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
8,Business reply mail Processing Centre,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.093023,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256


## create the new dataframe and display the top 10 venues for each neighborhood

In [132]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Neighbourhood']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighbourhoods_venues_sorted.iloc[ind, 1:] = row_categories_sorted.index.values[0:num_top_venues]


# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
1,"Alderwood, Long Branch",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
3,Bayview Village,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
5,Berczy Park,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
6,"Birch Cliff, Cliffside West",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
7,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
8,Business reply mail Processing Centre,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
9,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall


## Cluster Neighbourhoods

In [133]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [136]:
# add clustering labels
toronto_merged = toronto_df_new
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall


## visualize clusters

In [138]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Borough'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine cluster

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
1,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
2,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
3,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
4,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
5,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
6,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
7,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
8,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
9,Scarborough,0,Coffee Shop,Art Gallery,Japanese Restaurant,Café,Sushi Restaurant,Gift Shop,Gastropub,French Restaurant,Donut Shop,Concert Hall
